# Procesamiento de los datos del Dataset bo222k-1173.xyz

In [1]:
import pickle

## Exploración de los datos

### Número de átomos por muestra

In [2]:
# Nombre del archivo
nombre_archivo = "bo222-200k-1173k.xyz"

#Encontrar número de nuestras
with open(nombre_archivo, "r") as archivo:
    lista_atomos_en_molecula=[]
    for linea in archivo:
        linea=linea.split()
        if len(linea)==1:
            lista_atomos_en_molecula.append(linea[0])
# Crear un conjunto de números únicos en la lista
numeros_unicos = set(lista_atomos_en_molecula)

# Iterar sobre los números únicos y contar cuántas veces aparecen en la lista
for numero in numeros_unicos:
    veces_que_aparece = lista_atomos_en_molecula.count(numero)
    print(f"{veces_que_aparece} muestras tienen {numero} átomos cada una.")

with open("temp/atomos-por-molecula.pkl", "wb") as archivo:
    pickle.dump(lista_atomos_en_molecula, archivo)

if len(numeros_unicos)==1:
    numero_atomos=int(numeros_unicos.pop())
    print("Todas las muestras tienen el mismo número de átomos.")
else:
    print("Las muestras tienen distintos números de átomos.")

20000 muestras tienen 222 átomos cada una.
Todas las muestras tienen el mismo número de átomos.


## Ejemplo de muestra

In [3]:
# Nombre del archivo
nombre_archivo = "bo222-200k-1173k.xyz"
numero_de_muestras_a_imprimir=2

with open(nombre_archivo, "r") as archivo:
     # Inicializamos un contador de muestras
    contador_muestras = 0
    # Iteramos sobre las líneas del archivo
    for linea in archivo:
        # Si encontramos una línea que comienza con "MD_Step", incrementamos el contador
        if linea.startswith(str(numero_atomos)):
            contador_muestras += 1
            # Imprimimos la información de la muestra
            print(f"Muestra {contador_muestras}:\n{linea.strip()}")  # strip() elimina los espacios en blanco al principio y al final
            for _ in range(numero_atomos+1):
                print(str(archivo.readline().strip()))  # Imprimimos las siguientes 4 líneas (coordinadas de átomos)

            # Salto de línea para separar las muestras en la impresión
            print()

            # Si ya hemos impreso las dos primeras muestras, salimos del bucle
            if contador_muestras == numero_de_muestras_a_imprimir:
                break


Muestra 1:
222
MD_Step: 0 Total_energy = -392098.0704895295
H        12.5382170395        -7.6180493863        -8.3430311347        -0.39898648772126744        0.01952690778826339        0.5445614804687664
O        11.5886714189        -7.9580544718        -6.7810444904        0.31135376709519236        -0.056585372324846304        -0.07864721833014225
H        3.3587227872        -9.7510008565        -9.363537438        0.36682549685425936        -0.30268877340151773        -0.2658061311968897
O        9.8455171785        -3.1027899525        -7.2826422674        -0.08007644326414802        0.4438071698564543        0.2724695877759438
H        8.3874793281        9.8250429989        -5.3336833903        -0.067055722804807        -0.23375081267784817        0.29892145906568657
O        2.0539681079        -8.9315728042        -8.4516504011        -0.15112827260809886        0.3293702030833735        -0.049309828596247024
O        2.3448380358        -3.0549670935        -8.1765046048  

Unicamente los datos correspondientes a la energía total del arreglo (meV), las posiciones(A) y fuerzas de los átomos son de nuestro intéres(meV/A).

## Procesamiento de los datos

### Importación de módulos

In [4]:
import numpy as np
import h5py
import os
import shutil

### Creación de la carpeta auxiliar temp

In [5]:
carpetas=["temp","datos-procesados"]
for nombre_carpeta in carpetas:
    if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
        shutil.rmtree(nombre_carpeta)
        os.mkdir(nombre_carpeta)
    else:
        os.mkdir(nombre_carpeta)

### Funciones 

In [6]:
#Intercamcio de elemento por su posición en la tabla periodica
def posicion_en_tabla_periodica(elemento_quimico):
    tabla_periodica = {
        "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10,
        "Na": 11, "Mg": 12, "Al": 13, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18,
        "K": 19, "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24, "Mn": 25, "Fe": 26, "Ni": 27, "Co": 28,
        "Cu": 29, "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36,
        "Rb": 37, "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43, "Ru": 44, "Rh": 45, "Pd": 46,
        "Ag": 47, "Cd": 48, "In": 49, "Sn": 50, "Sb": 51, "Te": 52, "I": 53, "Xe": 54,
        "Cs": 55, "Ba": 56, "La": 57, "Hf": 72, "Ta": 73, "W": 74, "Re": 75, "Os": 76, "Ir": 77, "Pt": 78,
        "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, "Bi": 83, "Th": 90, "Pa": 91, "U": 92, "Np": 93, "Pu": 94,
        "Am": 95, "Cm": 96, "Bk": 97, "Cf": 98, "Es": 99, "Fm": 100
    }

    elemento = elemento_quimico.capitalize()
    if elemento in tabla_periodica:
        return tabla_periodica[elemento]
    else:
        return None

In [7]:
# energias atomicas
def atom_energy(elemento_quimico):
    energias_atomicas = {
        1:-0.500272784191*27.21138602,
        7:-242.365764213*27.21138602,
        8:-74.9555225243*27.21138602,
        13:-242.365764213*27.21138602
    }

    elemento = int(elemento_quimico)
    if elemento in energias_atomicas:
        return energias_atomicas[elemento]
    else:
        return None

### Guardar los datos seleccionados en archivos .data

In [8]:
# Nombre del archivo
nombre_archivo = "bo222-200k-1173k.xyz"
with open(nombre_archivo, "r") as archivo:
    # Iteramos sobre las líneas del archivo
    for linea in archivo:
        #Convertimos la linea en una lista
        linea=linea.split()
        # Si la longitud de la linea es 1, se trata del numero de átomos y el comienzo de la muestra
        if len(linea)==1:
            # inicialización de numeros atomicos
            atomos_muestra=[]
            numero_atomos_muestra=int(linea[0])
            # Guardado y Procesamiento de los datos de una muestra
            for numero_linea_en_muestra in range(numero_atomos_muestra+1):
                # Seleccionamos la siguiente linea
                linea_en_muestra=archivo.readline().strip()
                # convertimos a una lista la linea seleccionadda
                linea_en_muestra=linea_en_muestra.split()
##################################################################################
                # Guardamos la energia total en una variable
                if len(linea_en_muestra)==5:
                    energia_total=float(linea_en_muestra[-1])
             
                # Guardado de las fuerzas y posiciones
                if len(linea_en_muestra)==7:
                    # procesamiento 
                    z=posicion_en_tabla_periodica(linea_en_muestra[0])
                    atomos_muestra.append(z)
                    n_type=[z]

                    n_pos=[linea_en_muestra[1],linea_en_muestra[2],linea_en_muestra[3]]

                    n_force=[linea_en_muestra[4],linea_en_muestra[5],linea_en_muestra[6]]

                    # Guardado
                    with open("temp/pos.data", "a") as file:
                        line = " ".join(str(elemento) for elemento in n_pos)
                        file.write(line + "\n")
                    with open("temp/type.data", "a") as file:
                        line = " ".join(str(elemento) for elemento in n_type)
                        file.write(line + "\n")
                    with open("temp/force.data", "a") as file:
                        line = " ".join(str(elemento) for elemento in n_force)
                        file.write(line + "\n")

##################################################################################
                if numero_linea_en_muestra==(numero_atomos_muestra):
                    # Aplicar la función atom_energy a cada elemento de la lista usando map
                    resultados = map(atom_energy, atomos_muestra)

                    # Sumar los resultados
                    suma_resultados = sum(resultados)

                    # Calcular el promedio dividiendo la suma por el número de átomos en la muestra
                    numero_de_atomos = len(atomos_muestra)
                    n_energy=(energia_total-suma_resultados) / numero_de_atomos
                    n_energy=[n_energy]
                    with open("temp/energy.data", "a") as file:
                        line = " ".join(str(elemento) for elemento in n_energy)
                        file.write(line + "\n")

### Cargar datos en numpy arrays

In [9]:
# Leer los datos del archivo "energy.data"
with open("temp/energy.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
energy = np.array(data)

# Leer los datos del archivo "type.data"
with open("temp/type.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
types = np.array(data)

# Leer los datos del archivo "pos.data"
with open("temp/pos.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
pos = np.array(data)

# Leer los datos del archivo "force.data"
with open("temp/force.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
forces = np.array(data)

print("\narray's creados")


array's creados


### Convertir los datos a las dimensiones correspondientes

In [10]:
# Acomodo de dimensiones
numero_muestras=20000
energy=np.reshape(energy,(numero_muestras))
pos=np.reshape(pos,(numero_muestras,numero_atomos,3))
types=np.reshape(types,(numero_muestras,numero_atomos))
forces=np.reshape(forces,(numero_muestras,numero_atomos,3))

print("\nDimensiones acomodadas")


Dimensiones acomodadas


### Guardar en archivo .h5 (sin fuerzas)

In [11]:
########################################################
filename = "datos-procesados/bo222-200k-1173k.h5"##
########################################################

f=h5py.File(filename, "w")

##############################
group_name = "bo222-200k-1173k"##
##############################

group = f.create_group(group_name)

# Guardar datos en el grupo
group.create_dataset("types", data=types)
group.create_dataset("pos", data=pos)
group.create_dataset("energy", data=energy)

f.close()

### Guardar en archivo .h5 (con fuerzas)

In [12]:
########################################################
filename = "datos-procesados/bo222-200k-1173k-f.h5"##
########################################################

f=h5py.File(filename, "w")

##############################
group_name = "bo222-200k-1173k"##
##############################

group = f.create_group(group_name)

# Guardar datos en el grupo
group.create_dataset("types", data=types)
group.create_dataset("pos", data=pos)
group.create_dataset("energy", data=energy)
group.create_dataset("forces", data=forces)

f.close()

### Eliminación de la carpeta auxiliar

In [13]:
nombre_carpeta="temp"
if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)